In [7]:
import pandas as pd

# Load the uploaded file
df = pd.read_csv('customer_support_tickets.csv')

# Preview structure
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

Shape: (8469, 17)
Columns: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


# New Section

In [15]:
df = df.rename(columns=lambda c: c.strip().lower() if isinstance(c, str) else c)

candidates = ['message', 'text', 'body', 'comment', 'ticket']
text_col = next((c for c in candidates if c in df.columns), None)
if text_col is None:
    text_col = df.select_dtypes(include='object').columns[0]
print("Using text column:", text_col)

import re
def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df[text_col].apply(clean_text)
df[['clean_message']].head()

Using text column: customer name


,clean_message
0,marisa obrien
1,jessica rios
2,christopher robbins
3,christina dillon
4,alexander carroll


# New Section

In [16]:
issue_keywords = {
    'payment': ['payment', 'card', 'transaction', 'charged', 'billing', 'invoice'],
    'login': ['login', 'sign in', 'signin', 'password', 'otp', 'account'],
    'delivery': ['delivery', 'tracking', 'shipped', 'courier', 'delay', 'arrive'],
    'refund': ['refund', 'reimburse', 'return', 'money back'],
    'bug': ['error', 'bug', 'crash', 'exception', 'not working', 'fail'],
    'general': []
}

priority_keywords = {
    'P0': ['account hacked', 'data breach', 'fraudulent charge', 'payment failed multiple times'],
    'P1': ['payment', 'refund', 'login', 'unable to login', 'password'],
    'P2': ['delivery', 'tracking', 'delay'],
    'P3': ['general', 'question', 'inquiry', 'how to']
}

def classify_issue(text):
    for label, kws in issue_keywords.items():
        for kw in kws:
            if kw in text:
                return label.upper()
    return 'GENERAL'

def assign_priority(text):
    for p, kws in priority_keywords.items():
        for kw in kws:
            if kw in text:
                return p
    issue = classify_issue(text)
    return {
        'PAYMENT': 'P1',
        'REFUND': 'P1',
        'LOGIN': 'P1',
        'DELIVERY': 'P2',
        'BUG': 'P1',
        'GENERAL': 'P3'
    }.get(issue, 'P3')

df['issue_type'] = df['clean_message'].apply(classify_issue)
df['priority'] = df['clean_message'].apply(assign_priority)
df[['clean_message', 'issue_type', 'priority']].head()

,clean_message,issue_type,priority
0,marisa obrien,GENERAL,P3
1,jessica rios,GENERAL,P3
2,christopher robbins,GENERAL,P3
3,christina dillon,GENERAL,P3
4,alexander carroll,GENERAL,P3


# New Section

# New Section

In [20]:
issue_counts = df['issue_type'].value_counts()
priority_counts = df['priority'].value_counts()

report = {
    'total_tickets': int(len(df)),
    'by_issue': issue_counts.to_dict(),
    'by_priority': priority_counts.to_dict()
}
report

output_csv = 'triage_output.csv'
df.to_csv(output_csv, index=False)

# New Section

In [22]:
from google.colab import files
files.download('triage_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>